In [87]:
import re 
import pandas as pd 
import mysql.connector
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import time

In [175]:
mydb = mysql.connector.connect(host='73.38.248.152', user='buspark', password='U@5p1r3!')

if (mydb):
    print("Connection Successful")

else:
    print("Connection Unsuccessful")

mycursor = mydb.cursor()

mycursor.execute("Show databases")

for db in mycursor:
    print(db)

Connection Successful
('information_schema',)
('civica_courtdocs',)
('wp_courtdocs',)
('wp_courtdocs_NORMALIZED',)


In [177]:
df_action_null = pd.read_sql("SELECT * FROM wp_courtdocs.cdocs_case_action_index as c_a_index \
                          WHERE c_a_index.action = ' ' and c_a_index.actor = ' ' LIMIT 0, 1000;"
                          ,con = mydb)
df_action_null = df_action_null[['actor','action','description']]
df_action_null.columns = ['Actor','Action','Description']
df_action_null.head()

,Actor,Action,Description
0,,,NOTICE: Citation on Petition for Formal Adjud...
1,,,Citation Filed; Served as Ordered
2,,,"An Interested Person,Johanna Soris, Asst. Atto..."
3,,,Letters of Authority for Personal Representative
4,,,NOTICE: Citation on Petition for Formal Adjud...


In [171]:
chunksize = 100000
chunks = []
chunk_no=0
for chunk in pd.read_csv('./actor_action_desc.csv', chunksize=chunksize):
    print(chunk_no)
    chunks.append(chunk)
    chunk_no+=1
    if chunk_no==2:
        break
        
df_case_action = pd.concat(chunks,ignore_index=True)
del chunks
df_case_action.columns = ['Actor','Action','Description']
df_case_action.head()

0
1


,Actor,Action,Description
0,Plaintiff,Application filed,Application filed on 04/27/2011 at Boston Muni...
1,Plaintiff,Appearance filed,"Appearance for Potfolio Recovery Associates, L..."
2,Plaintiff,Application filed,Supplementary process application filed; filin...
3,Court,Event scheduled,HEARING SCHEDULED for 06/09/2011 10:00 AM.
4,Court,Return of Service,Return of service on complaint and summons to ...


In [257]:
def get_action_match_map():
    match_map = {
        # This takes precedence over other events 
        'Event cancelled' : [
            '/(Event|Trial|Session|review) scheduled .{0,50} (cancelled|canceled)/', 
            '/(Motion|DISMISSAL|conference|review).{0,50}SCHEDULED.{0,50}(cancelled|canceled)/',
            '/^PRH FOR .{5,12} CANCELLED /',

        ],
        'Acknowledgement of service' : [
            '/^Acknowledge?ment of service/',
        ],
        'Affidavit in support of motion' : [
            '/Affidavit of [\s\S]{1,50} in support of motion to/'
        ],
        'Affidavit filed' : [
            '/^Affidavit of [\s\S]{1,100} filed\.?/'
        ],
        'Affidavit' : [
            '/^Affidavit that/'
            
        ],
        'Agreement for Judgment' : [
            '/Agreement for judge?ment /' ,
            '/^Agreement filed/',
            '/Agreement for Judge?ment filed/',
            '/^.{5,40} Agreement Filed/',
            '/^Agreement/',

        ],
        'Answer filed' : [
            '/Answer filed by /'	,
            '/Answer of [\s\S]{2,55}/',
            'Answer to trustee summons filed ',
            'Defendant responded to initial complaint',
            '/^no ability to pay.Not working/',
            'Answer to third-party cross-claim',
        ],
        'Answer to counterclaim filed' : [
            '/Answer to (cross-?|counter-?) ?claim of[\s\S]*filed by/',
        ],

        'Amended Answer filed' : [
            'Amended answer filed by',
        ],
        'Appearance filed' : [
            '/Appearance for .{2,110} filed/',
            'Appearance filed for ',
            '/Appearance as substitute counsel .* filed/'

        ],
        'Application filed' : [
            'application filed' ,

        ],


        'Appointment of agent filed' : [  # Probate
            'Appointment of agent',

        ],
        'Arbitration' : [
            'Parties have agreed to Arbitration',
        ],
        'Assent':[
          'Assent',
          '/^Assent of [a-zA-Z-.\s]+$/',
          '/^[0-9]+ Assents'
        ],
        'Assent And Waiver Of Notice filed' : [
            '/An Interested Person[\s\S]*Filed MPC 455 to ?Assent And Waiver Of Notice/',
        ],
        'Assent and waiver': [
            '/^Assent and waiver of/'
        ],
        'Bankruptcy stay of proceedings' : [
            '/Bankruptcy stay of proceedings[\s\S]*has filed bankru(ptc|TCP)y/',
            '/^Misc Entry:[\s\S]*Notice of Bankruptcy Case Filing/'
        ],
        'Bond approved' : [
            '/^Bond with(out)? surety?(ies)? approved/',
            '/^Bond of (Trustee|personal representative) Approved[\s\S]{0,9}/',
            '/^Bond of [\s\S]{0,90} ?Approved[\s\S]{0,32}/'
        ],

        'Bond with sureties filed' : [
            'Bond with sureties',
        ],
        'Bond without sureties filed' : [
            'Bond without sureties',
        ],

        'Capias' : [
            'Capias issued'	,

        ],
        'Capias expired' : [
            'Capias returned expired',
        ],
        'Case inactivated' : [
            'Case Inactivated',
        ],
        'Case management conference' : [
            '/Case management conference[\s\S]*held/'
        ],
        'Capias returned' : [
            '/^Capias returned\.?/'  
        ],
        'Capias returned unserved' : [
            '/^Capias returned unserved/',
        ],

        # Probate

        'Certificate of Death filed' : [
            'Certificate of Death',

        ],
        'Change of address' : [
            '/^Change of address for /',
        ],
        'Complaint filed' : [
            '/Complaint .* civil action cover sheet filed/',
            'Plff filed complaint', 
            'complaint file', 
            'complaint filed'
        ],

        'Continuance' : [
            '/automatically continued until/',
            'CONTINUED FOR'	,
            'CONTINUED Until',
            '/ TO BE CONTINUED GENERALLY AT /',
            '/pltff ?(present)? deft no show.{0,100} cont til /',
            '/FIRST APPEARA?NCE IN TRIAL SESSI?O?N SCHEDULED.*RESCHEDULED/',
            '/.{0,20}rescheduled because .{0,20}/',
            '/ CONT FOR PAYMENT TIL /',
            '/up to date with payments cont[\s\S]{0,8}til/',
            '/ yet cont til /',
            '/ conts payment review til/',
            '/both parties present cont [0-9]+\/[0-9]+/',
            '/^(cont\.?|continue) generally ?, for settlement.*payment to clear/',
            'no one told him of the cont ',
            '/Motion (to continue|for continuance) ?[\s\S]* filed by[\s\S]*and assented to by other/' ,
            '/^Stipulation  ?of parties to continue case\.?/',
            '/^DEFENDANT IS PAYING\. ? ?CONTINUED GENERALLY/',
            '/^[\s\S]{0,25}continue generally per[\s\S]{0,25}/',
            '/Off list and continued[\s\S]{0,5}generally\.?/',
            '/^[\s\S]{5,16}? ?PRH CONTINUED TO/',
            '/^PRH (FOR|FROM|ON) [\s\S]{5,16} (CONTINUED|DEFERR?ED)/',
            '/^PRH FOR [\s\S]{5,16}TAKEN OFF LIST AT/',
            '/CONTINUE PRH TO [\s\S]{5,16}/',
        ],
        'Copy of will filed' : [
            '/^Will/',
            '/^Authenticated Copy of Will and Appointment[\s\S]{0,5}?(along with two codicils)?/',
            '/^A? ?(Certi?fied )?Copy of will[ 0-9\/\-]{0,20}/',
            '/^A? ?(Certi?fied )?Copy of will and appointment/',

            '/^A? ?(Certi?fied )?Copy of will \(?(original )?(filed)?[\s\S]{0,20}/',
            '/^A? ?(Certi?fied )?Copy of appointment and will/',
            '/^A? ?(Certi?fied )?Copy of will,? ?dated/',
            '/^A? ?(Certi?fied )?Copy of will,? ?[\s\S]{0,30}pa?ge?s/',
            '/^A? ?(Certi?fied|Authenticated)? ?Copy of will from[\s\S]{0,30}/',
            '/^A? ?(Certi?fied )?Photocopy of will[\s\S]{0,40}/',
            '/^A? ?Copy of Will with Alterations/',
            '/^Unsigned copy of will[\s\S]{0,40}/',

            '/^Will( [\s\S]{1,8})?/',
            '/^Will ? ?\( ?copy of ?\)/',
            '/^Will, dated .{2,20}/',

            'Will, ( entered in error ) see event #4 - authenticated copy of will',
        ],
        'Corporate disclosure statement filed' : [
            'Corporate disclosure statement filed',

        ],
        'Counsel added' : [
            '/added as .{0,20} Counsel/' ,
            '/Attorney.{6,30}representing.{6,}as of/',
            '/On this date .*, Esq\.?[\s\S]{0,25} added for /',
        ],
        'Counsel withdrawal' : [
                '/dismissed\/withdrawn as .{0,20} Counsel/',
                'Motion to withdraw as counsel filed',
                '/Attorney[\s\S]*dismissed\/withdrawn/',
        ],
        'Counterclaim filed' : [
            '/^Counterclaim filed by /',
            '/^ ?[\s\S]{3,55}\'s[\s\S]{0,4}Counterclaim[\s\S]{0,5}/'
        ],
        'Debtor before the court' : [
            'Judgment debtor before the court',
        ],
        'Decree and Order of Formal Probate' : [
            '/^Decree and Order of Formal Probate/',
        ],
        'Default' : [
            '/^.{0,15}Judgment (in SP)? ?by default/',
            '/^.{0,15}Default entered against /' ,
            'Judgment debtor failed to appear and was defaulted',
            'pltff present deft no show default judgment',
            'Default Judgment entered',
            '/^Defendant defaulted at /'
        ],
        'Default Removed' : [
            '/^.{0,15}Default removed against/'
        ],
        'Demand for Jury Trial' : [
            '/^Jury trial claim on all issues/',
            '/^Jury claim of /',
            'Jury Demand filed by ',

        ],
        'Deposit in Court' : [
            '/^Deposite?d? in court.{0,10}by /',
        ],
        'Dismissal' : [
            'DISM PAID IN FULL',
            '/Dismissed[\s\S]*PAID IN FULL/',
            '/^Supplementary process dismissed /',
            '/^Judgment of dismissal /',	
            '/CASE DISMISSE?D?\.? ? ?/',
            '/case[\s\S]{0,5}Dismissed at the request of/',
            'Notice of voluntary dismissal',
            'dismissed by agreement of parties',
            'Parties agree to dismiss case upon payment',
            '/^.{0,20}NPP ?- ?DISM\.?/',
            '/^.{0,12}Neither Party Present,? ?off list/',
        ],
        'Entry of Action filed' : [
            '/^.{0,20}Entry of action filed/',
        ],

        'Event resulted' : [
            '/^Event resulted[\s\S]*Result:/',

        ],
        'Event scheduled' : [
            '/^Event scheduled[\s\S]*Date/',
            'hearing scheduled',
            '/^(Motion|Payment review|status review) scheduled/',

            'PRETRIAL CONFERENCE SCHEDULED for',
            'TRIAL SCHEDULED',
            'CASE MANAGEMENT CONFERENCE SCHEDULED',
            '/FIRST APPEARA?NCE IN TRIAL SESSI?O?N SCHEDULED/',
            '/^Scheduled[\s\S]{0,10}Event:/',

        ],
        'Execution' : [
            '/^Execution (Writ )?issued/',
            '/^Execution Writ for possession [\s\S]*issued[\s\S]*sent to/',
        ],
        'Execution satisfied' : [
            '/^Execution Writ (for possession ?)?(returned )?(fully )?satisfied/',
        ],
        'Execution unsatisfied' : [
            '/^Execution Writ (returned )?unsatisfied/',
        ],

        'Ex Parte hearing' : [
            '/^.{0,20}Ex parte hearing held/'
        ],


        'Fee paid' : [
            '/Filing fee .{1,100} paid/',
            '/^Summary Process:[\s\S]*MGL[\s\S]*Receipt:.*Date/',
            '/^Surcharge[\s\S]*MGL[\s\S]*Receipt:.*Date/',
        ],
        'Fee due'  : [
            'Filing Fee due',
            'Filing Fee Surcharge due',
            '/^Small Claims Filing Fee [\s\S]{0,50}due/',
        ],
        'Fee waived' : [
            'filing fee waived on finding of',
            'Filing fee and surcharge waived on'
        ],
        'Hearing' : [
            '/.{4,15} 4th session both parties present.{20,}/',
            '/.{4,15} 4th session.*[0-9]{2}(am|pm).{20,}/' ,
            '/9:28 offlist deft arrived @ 9:31; pltff not pres;/',
            '/rm 125 deft arrived @ 11:06 paid \500/',
            'both parties present deft has not been living',
            'both parties present; deft brought letter',
            '/start \/ stop .{0,12}both parties present deft in custody 1st session/',
            '/ session .{0,20}both parties present[\s\S]/',

            '/child care center (pltff|deft) present (pltff|deft) show/',
            '/^.{5,12}deft present only\.?/',
            '/^bpp present /',
            'th session, deft. only present',
            '6-20-2011 @ 4:30 p.m. Asst. Clerk-Magistrate',
            'START 1046 / STOP 2165 / BOTH PARTIES PRESENT',
            '3/21/11 plaintiff not present, Deft. only present',
            'child care center pltff present deft no show',
            '/Room [0-9]+, pltff\.? only present, deft\.? not present/',
            '/Motion for attachment by trustee process [\s\S]{0,30}heard [\s\S]{0,40}Tape [0-9]th session/',
        ],
        'Interpreter requested' : [
            '/^.{0,12}Interpreter requested/',
        ],
        'Judgment' : [
            '/^Judgment issued/',
            '/^Judgment by Default for /',
            '/^Judgment for /' ,
            '/^Judgment Entered:/',
            'Judgment for pltff',
            'judgment for deft',
            '/Order for Judgment[\s\S]*findings for/',
        ],
        'Judgment amended' : [
            '/Amended Default Judgment.*issued/',
        ],
        'Judgment debtor able to pay' : [
            '/Judgment debtor.*(found|stipulated).* to be.* able to pay on judgment/',
        ],
        'Judgment debtor unable to pay' : [
            '/Judgment debtor.*(found|stipulated).* to be.* unable to pay on judgment/',
        ],
        'Judgment vacated' : [
            '/^Judgment entered on .{2,20} vacated/'
        ],
        'Mediation' : [
            '/^Mediation held\.?/'	 
        ],

        'Memorandum filed' : [
            '/^Memorandum filed by /'	 
        ],
        'Military affidavit filed' : [
            '/^Military affidavit filed ?/',
            '/Military Service and (Rule 10 )?Affidavit/',
            'Affidavit as to Military Service',
        ],
        'Must file to avoid judgment' : [
            '/^(Deft|Pltff)\.? to file [\s\S]{5,120}or judgment[\s\S]{1,10}entered/',
        ],

        'Motion allowed' : [
            '/^Motion  ?(to )? ?(acc?ept|allow|amend|appoint|approve|assess|compel|continue|dismiss|execution|issue)[\s\S]*allowed/',
            '/Motion  ?(for|to) [\s\S]*(is|and)? allowed\.?.{0,170}/' ,
            '/Motion  ?(for|to) .* heard at which .* were present and ALLOWED/' , 
            '/Motion  ?for new capias[\s\S]* allowed/',
            '/Motion  ?for summary judgment allowed/',
            '/^Motion PHOTOCOPY [\s\S]*Allowed/',
            'pltff present deft no show mtoion allowed capias',
            'Structured settlement transfer approved',
            '/^Attachment by trustee process[\s\S]{0,50} allowed against/',

        ],
        'Motion denied' : [
            '/(Misc entry:?)? ?Mot(ion)?  ?(for|to) [\s\S]*(is|and)? ?denied\.?/' ,
            '/^Motion  ?(to )? ?(acc?ept|allow|amend|appoint|approve|assess|compel|continue|dismiss|execution|issue)[\s\S]*denied/',

        ],
        'Motion withdrawn' : [
            '/^.{0,25}Motion (transfer|to|for) [\s\S]* waived or withdrawn/',
            '/^.{0,25}Motion (transfer|to|for) [\s\S]* withdrawn by moving/',

        ],
        'Motion filed' : [
            '\^Motion filed/',
            '\^Motion  filed/'
        ],

        'Motion to accept copy of will' : [
            '/^Motion to (allow|approve|ac?cept) (photo)?copy of will( as original)?/',

        ],


        'Motion to amend judgment' : [
            '/^Motion ? ?to amend judgment and execution filed/',
            '/^Motion  ?to Amend Default Judgment.*filed/',

        ],

        'Motion to amend Voluntary Administration Statement' : [
            '/^Motion to amend voluntary administration/'
        ],
        'Motion to appoint special process server' : [
            '/^Motion to appoint special process server/',
        ],
        'Motion to approve transfer of structured settlement' : [
            '/Motion transfer of structured settlement ? ?filed/',				

        ],
        # These motions should appear beneath allowed/withdrawn motions above
        'Motion to attach by trustee process' : [
            '/^(Ex parte )?motion for attachment by trustee process/',
            '/Motion ? ?to attach wages and for successive service on trustee/',
        ],

        'Motion to assess attorney fees' : [
            '/^(on .{0,16})?Motion  ?(to|for)? ?assess(ment)? (of )?attorney(\'s)? fees/'
        ],

        'Motion to assess damages' : [
            '/^(on .{0,16})?Motion ? ?(to|for)? ?assess(ment)? (of )?damages/'
        ],
        'Motion to compel discovery' : [
            '/^Motion  ?to compel discovery/',
            '/^Motion  ?to compel production of documents/',
        ],
        'Motion for continuance' : [
            '/^Motion ? for ? continuance ? ?[\s\S]*filed by/',
            '/^Motion to continue trial filed/',
        ],

        'Motion for default judgment' : [
            '/^.{0,16}Application[\s\S]*(for|to enter) default[\s\S]* against /',
            '/Motion for default judgment and req to assess damages filed/',
            'Motion for Entry of Default Judgment filed',
            '/Motion to default (&|and) charge trustee/',
        ],


        'Motion to dismiss counterclaim' : [
            '/^Motion ? ?to Dismiss Counterclaim/'
        ],
        'Motion for exemption from time standards' : [
            '/^Motion  ?t?o? ? ?(request )?for exemption from time standards filed by/',
        ],
        'Motion for final judgment for relief' : [
            '/^Motion  ?for final judgment for relief[\s\S]{0,50}filed by/',
        ],

        'Motion to inspect records' : [
            '/^Motion  ?for Inspection and Copying of[\s\S]*records/'
        ],
        'Motion to issue execution' : [
            '/Motion (to )? ?issue execution filed/',
            '/Motion to Issue Execution Late filed/',
            '/Application.{0,26} for Execution Writ/',
        ],
        'Motion for judgment on the pleadings' : [
            '/^.{0,15}Motion  ?for judgment on the pleadings [\s\S]*filed/',
        ],
        'Motion to lift garnishment' : [
            'Motion to lift garnishment filed',
        ],
        'Motion for minor settlement' : [
            '/^Motion minor settlement/'
        ],


        'Motion for new capias' : [
            '/Motion *(for new|lost) capias[\s\S]*filed by/',
            '/^Motion expired capias/',
            '/Motion new capias ? ?filed by/',
            '/Motion new capias  ?made in open court/',
            '/Motion (to )?Renew capias filed/',

        ],

        'Motion to remove default' : [
            '/^Motion ? ?to remove default and file answer late/',
            '/^Motion ? ?to remove default[\s\S]*filed by/',
            '/^Motion by ? ?[\s\S]{1,30} ? ?to remove default/',
        ],
        'Motion to stay execution' : [
            'Motion to further stay Execution filed',
        ],
        'Motion to strike answer' : [
            '/Motion to strike .*answer /',
        ],
        'Motion to Substitute a Party' : [
            '/^Motion.*to substitute.*party/',
        ],
        'Motion for successive service of trustee summons' : [
            '/^Motion for successive service of trustee summons/'
        ],
        'Motion for summary judgment' : [
            '/^.{0,70}Motion for summary judgment/',
        ],
        'Motion to serve trustee summons' : [
            '/^Motion[\s\S]{0,4}to serve (copy of )?trustee summons/',
        ],
        'Motion to transfer' : [
            '/^Motion to transfer .* filed/',
            '/Copy of Motion to transfer [\s\S]*filed/',
        ],
        'Motion to vacate dismissal' : [
            '/^Motion to vacate dismissal [\s\S]*filed by/',
        ],
        'Motion to vacate judgment' : [
            '/^Motion ? ?to vacate judgment[\s\S]*filed by/',
            '/^Motion ? ?vacated? judgment[\s\S]*filed by/',
        ],



        'Notice of appeal'	 : [
            'Notice of appeal for trial',
        ],
        'Notice of claim' : [
            '/^Notice of claim:[\s\S]{1,20}/'
        ],
        'Notice of dismissal' : [
            '/Notice of dismissal .*filed by /',
            'Notice of dismissal filed',
            '/Stipulation of dismissal.*filed/',
        ],
        'Notice of potential default' : [
            '/parties notified[\s\S]*subjected to defaulting[\s\S]{0,20}failure to answer/',
        ],
        'Notice of potential dismissal' : [
            'Notice of Potential Dismissal',
            '/^DISMISSAL .{0,50}potentially applicable/',
            '/^DISMISSAL FOR FAILURE TO ACT[\s\S]*potentially applicable on/'
        ],

        'Notice of intent to offer medical records' : [
            '/^Affidavit [\s\S]*(under|pursuant to).*233.*79G/',
            '/^Affidavit [\s\S]*intent[ion]{3}? to offer medical[\s\S]*records/',
            '/^Affidavit of [\s\S]* for medical Recor?ds from [\s\S]',
            'Intention to offer certified Medical Records',
            '/Affidavit in compliance with [\s\S]*233[\s\S]*79G/',
            '/Notice of Intention to offer [\s\S]*Medical Records/',

        ],
        'Notice sent' : [
            '/^Notice concerning next court event.*sent to parties\.?/',
            '/^Notice of next event sent/',
            'Notice of case management conference sent',
            'Notice of pretrial conf',
            '/^Notice of judgment sent to parties/',
            '/^Notice sent to parties\.?([\s\S]{0,10}A? ?Noti?ce (sent )?to the Parties[\s\S]{0,20}sent[\s\S]{0,100})?/',
            '/^Notice of court action on .{1,15} sent to parties/',
            '/^Notice of Default sent to parties/',
            '/^notice of PTC? (mailed|sent)/',
        ],

        'Notice of trial' : [
            'notice of trial issued',
        ],
        'Notice to quit filed' : [

        ],
        'No capias returned' : [
            'No Capias returned' , 
        ],
        'No parties present' : [
            '/^NPP.{0,3}/',
            '/^.{0,12}neither party present\.?/',
            'neither party present offlist',
            'NEITHER PARTY IN COURT',

        ],
        'Opposition to motion' : [
            '/Opposition filed by[\s\S]{1,40}to Motion/',
            '/Opposition to Motion filed by[\s\S]{1,40}/',
        ],
        'Order for Informal Probate' : [
            '/^Order for Informal Probate of Will/',
        ],
        'Order Nisi for Dismissal' : [
            '/^Order nisi for judgment of dismissal/',
        ],

        # Unknown actual order, terse (nisi = unless)
        'Order Nisi' : [
            '/^Order:? NISI;? ?parties notified[\s\S]{0,7}/'
        ],
        'Original note filed' : [
            '/^Original note or other paper filed/',
        ],
        'Payment order' : [
            '/^.{0,20}Payment order/', 
            '/^Order, you have until.{2,20}to place the amount of /',
        ],
        'Payment review' : [
            'Payment Review scheduled for[\s\S]*has[\s\S]*been[\s\S]*resulted', 
            '/^Payment review held/',
            '/ Sess\. BPP before .{1,20} for P\.R\./', 
            '/ session for payment review/',
            '/amount cont for further payment review /',
        ],


        # Probate matters #

        'Petition for Administration' : [
            '/^Petition for Administration ?(DBN\/)?(CTA)?/',
            'Petition for Appointment of Public Administrator'
        ],
        'Petition for Allowance of Account'   : ['Petition for Allowance of Account'],
        'Petition for Appointment of Trustee' : ['Petition for Appointment of Trustee'],
        'Petition for Formal Probate' : ['Petition for Formal Probate'],
        'Petition for Informal Probate' : ['/^Petition for Informal/'],
        'Petition for Order of Complete Settlement' : ['Petition for Order of Complete Settlement'],
        'Petition for Probate of Foreign Will' : [],
        'Petition for Probate of Will and Appointment of Executor' : [],
        'Petition for Removal of Personal Representative' : [],
        'Petition for Sale of Real Estate' : ['Petition for Sale of Real Estate'],		
        'Petition for Special Administration' : [],

        # END probate matters #


        'Pre-Trial conference' : [

            '/Pretrial conference held/',
        ],
        'Pre-Trial Memorandum' : [

            '/Pre-Trial Memorandum filed/',
        ],
        'Release of All Demands and Assent to Account' : [
            '/^Release of All Demands and Assent to Account/',
        ],
        'Request for admissions filed' : [
            '/^Request for admissions/',
        ],
        'Request for continuance' : [
            '/ requests that case be continued/',
            '/^Parties jointly request that case be continued/',
            '/^.{0,25}requested a new hearing date/',
        ],

        'Request for Temporary Restraining Order' : [
            ' files request for TRO ',
        ],
        'Request to not dismiss' : [
            '/^Pla?i?n?t?i?ff\.?  ?reports that case is active and requests that it[\s\S]*not be dismissed/'
        ],

        'Return of Service' : [
            '/^Return of service/'	 
        ],
        'Satisfaction of judgment' : [
            'Satisfaction of judgment filed',
            'pltff present reports judgment satisfied', 
            'present judgment is satisfied',
            'pltff reports judgment satisfied',
        ],
        'Settlement agreement' : [
            '/MEDIATED SETTLEMENT[\s\S]*AGREEMENT REACHED/',
            'both parties present mediated (settlement|agreement)',
            'Settlement approved between ', 
            '/^Settlement reported by parties/',

        ],
        'Statement of small claim filed' : [
            '/Statement of small claims? entered/',
            '/statement of small claims? filed/',
        ],
        'Statement of damages filed' : [
            'statement of damages filed by' ,
        ],
        'Statement of Voluntary Administration filed' : [
            '/^Statement of Voluntary Administration[\s\S]'
        ],
        'Status Review' : [

            '/^.{0,15}Status review held/',
        ],
        'Suggestion of bankruptcy filed' : [ # in real terms this is a NOTICE of bankruptcy
            '/Suggestion of bank?ruptcy .{0,70}filed/',
            'Notice of Chapter 7 Meeting filed from Bankruptcy Court',
        ],

        'Suggestion of death filed' : [
            '/^Suggestion of Death of[\s\S]{0,40}/'
        ],

        # Housing
        'Summary Process Summons and Complaint filed' : [
            '/^SP Summons and Complaint[\s\S]*(rent|cause)/',
        ],

        'Summons' : [
            'summons to ',	 
            'summons issued for ',
        ],
        'System Notes' : [
            'This Case Converted from',

        ],
        'Termination notice filed' : [
            '/^Termination notices? filed[\s\S]{0,40}/'
        ],
        'Third-party cross-claim filed' : [
            '/^Third-party cross-claim filed/'
        ],
        'Transfer of case' : [
            '/^Case received at .{0,100} Court.{0,30}from .{0,100} Court\.?/',
            '/^Case transferred out from [\s\S]* to /',
            '/^Case ordered transferred to /',
            '/^Misc\.? Entry:? ?Case transferred to/',
            '/^transferr?ed from .{4,20}/',
        ],
        'Trial' : [
            '/.{0,14}First appearance in trial session held/',
            'Jury trial held',
            '/both parties present .{0,5}session.{0,15} trial on the merits begins/',
        ],
        'UCC filed' : [
            '/Uni?fo?rm Counse?l Certi?fi?ca?tion (form )?filed by/',
        ],
        'Under Advisement' : [
            '/Motions? Taken under advisement/',
            '/Taken under advisement\.?/'
        ],
        'Verification of Defendant\'s Address filed' : [
            '/Veri?fi?cation of Def(endan)?t.{0,3} Address (form )?filed/',
        ],
        'Waiver of Attorneys Fees filed' : [
            '/ files waiver of attorneys fees\.?/',
        ],
        'Waiver of Award of Pre-Judgment Interest filed' : [
            '/^Waiver of Award of Pre-Judgment Interest filed/'
        ],
        'Waiver of jury trial' : [
            'Waiver of jury filed by ',
        ],
        'Witness list filed' : [
            '/^.{3,12} Witness List filed/',

        ],


        # Catch blobs of notes
        'Misc Notes' : [

            '/^deft\.? paid money order.{5,15}and will pay.{5,60}/',
            'Unattested photocopies of court documents.',
            'Blank Summons MGL 262 section 4b',
            '/^Certificate of Orders, Decrees, Rulings, Judgments or Other Proceeding/',
            '/^Misc Entry: ?[\s\S]{300,}/',
            '/^Misc Entry: ?account closed/',
            '/^Misc Entry: ?pltff\'s calls/',
            '/^Misc Entry: ?deft drops off /',
            '/^Misc Entry: ?DEFT SENDS /',
            '/^Misc Entry: ?OFF LIST/',
            'Misc Entry: Motion taken off list',
            '/^Misc Entry: \[0-9\.]+ money order rec/',
            '/^Misc Entry: ?Deft\.? called and spoke/',
            '/^REC\'D COPY DEFT WILL MAIL PAYMENT TO PLTFF.{0,60}/',
            'PLAINTIFF TO FILE MOTION TO ISSUE EXECUTION IF NEEDED BEFORE',
            '/^deft\. called would like to speak with pltff/',
            '/^no entry fee was received.*sent letter to pltff[\s\S]*request them[\s\S]*forward the payment/',

            'dismissed request of pltff in court acm john clough',

            '/^no action was given to clerk by covering atty/',
        ]


    }
    
    return match_map




In [241]:
class Normalizer():
    def __init__(self, model_dir):
        self.model_dir = model_dir
        self.action_match_map = get_action_match_map()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.tokenizer,self.model = self.load_tokenizer_and_model()

    def extract_action_regex(self,description):    
        out = ''
        description = description.lower()
        for normalized_val ,vals in self.action_match_map.items():
            # Already matches normalized value exactly
            if description == normalized_val.lower():
                out = normalized_val
                break
            for v in vals:
                v = v.lower()
                try:
                    # Regex
                    if v.startswith('/'):
                        
                        v2 = v[1:]
                        if v2.endswith('/'):
                            v2 = v2[:-1]
                        if re.match(v2,description):
                            out = normalized_val
                            break
                            
                    else: # simple
                        temp = min([len(description),90])
                        if description[0:temp].find(v) != -1:
                            out = normalized_val
                            break 
                except:
                    continue
        return out

    def load_tokenizer_and_model(self):

        tokenizer = AutoTokenizer.from_pretrained("deep-learning-analytics/wikihow-t5-small")
        model = AutoModelWithLMHead.from_pretrained(self.model_dir).to(self.device)
        return tokenizer,model


    def extract_action_model(self,description):
        
        preprocess_description = description.strip().replace("\n","")
        tokenized_description = self.tokenizer.encode(preprocess_description, return_tensors="pt").to(self.device)
        
        action_ids = self.model.generate(
              tokenized_description,
              max_length=10, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
          )

        out = self.tokenizer.decode(action_ids[0], skip_special_tokens=True)
        return out 
    
    def extract_action(self,description):
        
        out = self.extract_action_regex(description)
        if out == "":
            out = self.extract_action_model(description)
            
        return out

In [258]:
model_dir = './pytorch_models/2/'
normalizer = Normalizer(model_dir)

In [261]:
df_temp = df_case_action.iloc[:1000]
df_temp['Predicted_Action'] = df_temp['Description'].apply(lambda x: normalizer.extract_action(x))

<ipython-input-261-d18b8d8618f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Predicted_Action'] = df_temp['Description'].apply(lambda x: normalizer.extract_action(x))


In [262]:
df_temp[df_temp['Action'] != df_temp['Predicted_Action']]

,Actor,Action,Description,Predicted_Action
4,Court,Return of Service,Return of service on complaint and summons to ...,Summons
9,Plaintiff,Notice of dismissal filed,Notice of dismissal of supplementary process w...,Notice of dismissal
11,Court,Capias expired,Capias returned expired and unserved. Defenda...,Capias returned
36,Plaintiff,Notice of dismissal filed,Notice of dismissal filed by P01 CLARENCE WIL...,Notice of dismissal
52,Court,Capias expired,Capias returned expired and unserved. New cap...,Capias returned
...,...,...,...,...
880,Plaintiff,Notice of dismissal filed,Stipulation of dismissal with prejudice filed ...,Notice of dismissal
923,Plaintiff,Fee paid,"Summary process entered; complaint, summons, t...",Agreement for Jud
925,Court,Continuance,FIRST APPEARNCE IN TRIAL SESSN SCHEDULED on 5/...,Event scheduled
953,Plaintiff,Notice of dismissal filed,Notice of dismissal of supplementary process w...,Notice of dismissal


In [259]:
t1 = time.time()
df_action_null['Predicted Action'] = df_action_null['Description'].apply(lambda x: normalizer.extract_action(x))
t2 = time.time()
print("Time taken:",t2-t1)

Time taken: 210.65151262283325


In [260]:
df_action_null.to_csv('./predicted_actions.csv',index=False)

In [243]:
desc = df_action_null.at[8,'Description']
action = normalizer.extract_action_regex(desc)
print(desc)
# print('\n\n Action:',action)
# r = '^Assent of [a-zA-Z-.\s]+$'
# print(re.match(r.lower(),desc.lower()))

Assent of Mary Jeanne Mullen
